In [3]:
%pip install -U datasets sentence-transformers pinecone-client tqdm

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from datasets import load_dataset

ytt = load_dataset(
    "pinecone/yt-transcriptions",
    split="train",
    revision="926a45"
)
ytt


/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset json (/Users/amzamani/.cache/huggingface/datasets/pinecone___json/pinecone--yt-transcriptions-4383a838c5c147e8/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


Dataset({
    features: ['video_id', 'text', 'start_second', 'end_second', 'url', 'title', 'thumbnail'],
    num_rows: 11298
})

In [5]:
for x in ytt:
    print(x)
    break


{'video_id': 'ZPewmEu7644', 'text': " hi this is Jeff Dean welcome to applications of deep neural networks of Washington University in this video we're going to look at how we can use ganz to generate additional training data for the latest on my a I course and projects click subscribe in the bell next to it to be notified of every new video Dan's have a wide array of uses beyond just the face generation that you", 'start_second': 0, 'end_second': 20, 'url': 'https://www.youtube.com/watch?v=ZPewmEu7644&t=0s', 'title': 'GANS for Semi-Supervised Learning in Keras (7.4)', 'thumbnail': 'https://i.ytimg.com/vi/ZPewmEu7644/maxresdefault.jpg'}


In [6]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

retriever = SentenceTransformer(
    'flax-sentence-embeddings/all_datasets_v3_mpnet-base'
)
retriever.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [7]:
embed_dim = retriever.get_sentence_embedding_dimension()
embed_dim

768

In [8]:
import pinecone

# get api key from app.pinecone.io
pinecone.init(
    api_key="4364c50c-b87b-4bdb-945a-18fbe9aff3b4",
    environment="us-west4-gcp"
)

# create index
pinecone.create_index(
    "youtube-search",
    dimension=embed_dim,
    metric="cosine"
)

# connect to new index
index = pinecone.Index("youtube-search")

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=UTF-8', 'date': 'Fri, 26 May 2023 16:28:05 GMT', 'x-envoy-upstream-service-time': '1133', 'content-length': '131', 'server': 'envoy'})
HTTP response body: The index exceeds the project quota of 1 pods by 1 pods. Upgrade your account or change the project settings to increase the quota.


In [9]:
index = pinecone.Index("youtube-search")

In [ ]:
from tqdm.auto import tqdm

docs = []  # this will store IDs, embeddings, and metadata

batch_size = 64

for i in tqdm(range(0, len(ytt), batch_size)):
    i_end = min(i+batch_size, len(ytt))
    # extract batch from YT transactions data
    batch = ytt[i:i_end]
    # encode batch of text
    embeds = retriever.encode(batch['text']).tolist()
    # each snippet needs a unique ID
    # we will merge video ID and start_seconds for this
    ids = [f"{x[0]}-{x[1]}" for x in zip(batch['video_id'], batch['start_second'])]
    # create metadata records
    meta = [{
        'video_id': x[0],
        'title': x[1],
        'text': x[2],
        'start_second': x[3],
        'end_second': x[4],
        'url': x[5],
        'thumbnail': x[6]
    } for x in zip(
        batch['video_id'],
        batch['title'],
        batch['text'],
        batch['start_second'],
        batch['end_second'],
        batch['url'],
        batch['thumbnail']
    )]
    # create list of (IDs, vectors, metadata) to upsert
    to_upsert = list(zip(ids, embeds, meta))
    # add to pinecone
    index.upsert(vectors=to_upsert)

100%|██████████| 177/177 [43:54<00:00, 14.88s/it] 


In [10]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 11298}},
 'total_vector_count': 11298}

In [11]:
query = "What is deep learning?"

xq = retriever.encode([query]).tolist()

In [17]:
xc = index.query(xq, top_k=5,
                 include_metadata=True)
# print(xc)
for context in xc['matches']:
    print(context['metadata']['text'], end="\n---\n")

 terms of optimization but what's the algorithm for updating the parameters or updating whatever the state of the network is and then the the last part is the the data set like how do you actually represent the world as it comes into your machine learning system so I think of deep learning as telling us something about what does the model look like and basically to qualify as deep I
---
 any theoretical components any theoretical things that you need to understand about deep learning can be sick later for that link again just watched the word doc file again in that I mentioned the link also the second channel is my channel because deep learning might be complete deep learning playlist that I have created is completely in order okay to the other
---
 under a rock for the last few years you have heard of the deep networks and how they have revolutionised computer vision and kind of the standard classic way of doing this is it's basically a classic supervised learning problem you are givi

In [12]:
index.describe_index_stats()


{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 11298}},
 'total_vector_count': 11298}